In [ ]:
# 耗时： 2.372464ms
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[16, 64, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806 = sch.get_loops(block=b80)
    sch.vectorize(l806)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815 = sch.get_loops(block=b81)
    sch.vectorize(l815)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l16)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 64, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
#耗时: 1.280608ms

def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[4, 256, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806 = sch.get_loops(block=b80)
    sch.vectorize(l806)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815 = sch.get_loops(block=b81)
    sch.vectorize(l815)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l16)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时: 1.941032ms
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[2, 512, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806 = sch.get_loops(block=b80)
    sch.vectorize(l806)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815 = sch.get_loops(block=b81)
    sch.vectorize(l815)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l16)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 512, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)


In [ ]:
# 耗时: 1.056472ms
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806 = sch.get_loops(block=b80)
    sch.vectorize(l806)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815 = sch.get_loops(block=b81)
    sch.vectorize(l815)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l16)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时: 1.855816ms
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[16, 256, 1]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806 = sch.get_loops(block=b80)
    sch.vectorize(l806)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815 = sch.get_loops(block=b81)
    sch.vectorize(l815)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l16)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时:  0.769808ms, 结果错误(拆分了reduce,降低单线程计算量, 没有做最终的reduce)
# 操作: 根据以上的实验，推测热点耗时在计算上: 同一个线程做reduce操作，一个线程内计算量为 11008 * 2 (multiple + add), 导致如何拆分都无法达到加速的效果
#       因此将reduce操作拆分到不同线程上, 降低单线程计算量, 实验显示耗时可以降低
#       @TODO 但是目前不知道该如何做最终的规约，即 如何计算 reduce = sum(*sub_reduces)
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 128, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    # v13, v14, v15 = sch.sample_perfect_tile(
    #     loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    # )
    v100, v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=4, max_innermost_factor=8, decision=[8, 43, 4, 8]
    )
    l100, l16, l17, l18 = sch.split(
        loop=l5, factors=[v100, v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l100, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.bind(loop=l100, thread_axis="threadIdx.y")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l100)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 128, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时: 1.168656ms
# 推测: 在可搜索空间内, 降低单线程计算量已经无法提升性能了
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[64, 16, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    # v13, v14, v15 = sch.sample_perfect_tile(
    #     loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    # )
    v100, v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=4, max_innermost_factor=8, decision=[43, 8, 4, 8]
    )
    l100, l16, l17, l18 = sch.split(
        loop=l5, factors=[v100, v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l100, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.bind(loop=l100, thread_axis="threadIdx.y")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l100)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 16, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时:0.907048ms, 无效实验
# 推测: 降低threadDim, 提升blockDim, 即划分更多的work group不能提升耗时了
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[16, 64, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    # 这个正好, weight 4*8 = 32 对应group_size = 32, 正好对应vload4的scale
    # v13, v14, v15 = sch.sample_perfect_tile(
    #     loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    # )
    v100, v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=4, max_innermost_factor=8, decision=[8, 43, 4, 8]
    )
    l100, l16, l17, l18 = sch.split(
        loop=l5, factors=[v100, v13, v14, v15], preserve_unit_iters=True
    )
    sch.reorder(l10, l11, l100, l16, l17, l18, l12)
    sch.bind(loop=l10, thread_axis="blockIdx.x")
    sch.bind(loop=l11, thread_axis="threadIdx.x")
    sch.bind(loop=l100, thread_axis="threadIdx.y")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b1, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l17, preserve_unit_loops=True, index=-1)
    print(sch.get_loops(block=b80))
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b1, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l16, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b1, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l11, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b1, loop=l100)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    l29, l30, l31, l32, l33 = sch.get_loops(block=b20)
    l34, l35, l36 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l33, factors=[None, 64, 8], preserve_unit_iters=True
    )
    sch.vectorize(loop=l36)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时:  0.717664ms  结果基本对齐
# 方法: 将reduce做拆分，最后合并结果
# 观测心得:  在计算时不使用vectorize 耗时约为5.x ms, 使用vectorize(4)后耗时降低为0.717664ms
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    # @TODO 8, 64, 8
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 128, 4]
    )
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=4, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    v161, v162 = sch.sample_perfect_tile(loop=l16, n=2, max_innermost_factor=8, decision=[8, 43])
    l161, _ = sch.split(l16, [v161, v162], preserve_unit_iters=True)
    # 将reduce操作按照l161做拆分，以便并行化. 其过程是: 1. 申请 [l161, 1, 1, 4096]个xxx_rf内存, 2. 将计算拆为l161个chunks分别计算, 3. 按照l161维度reduce, 得到最终的输出
    sch.rfactor(l161, 0)
    ## 此时b1还是block('matmul'), 但是计算内容变为rf的合并操作(即: reduce = sum(sub_reduces))
    ## 此时拆分的reduce计算变为 block('matmul_rf')
    b2 = sch.get_block(name="matmul_rf", func_name = "main")
    l01, l02, l03, l04, l05, l06, l07 = sch.get_loops(block=b2)
    # 8, 128, 8, 43, 4, 8, 4
    sch.reorder(l01, l02, l04, l05, l06, l07, l03)
    b21 = sch.cache_write(block = b2, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block = b21, loop=l04, preserve_unit_loops=True, index=-1)
    print(sch.get(b21))
    lb311, lb312, lb313, lb314, lb315,lb316, lb317 = sch.get_loops(block=b2)
    sch.vectorize(lb317)
    lb211, lb212, lb213, lb214, lb215,lb216, lb217 = sch.get_loops(block=b21)
    sch.vectorize(lb217)
    # print(sch.mod.script())
    sch.bind(loop=l01, thread_axis="blockIdx.x")
    sch.bind(loop=l02, thread_axis="threadIdx.x")
    sch.bind(loop=l04, thread_axis="threadIdx.y")
    sch.compute_inline(block=b0)
    # 计算的中间结果: var_matmul_intermediate_local, shape[1,1,4096]
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block=b19, loop=l11, preserve_unit_loops=True, index=-1)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b2, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=l06, preserve_unit_loops=True, index=-1)
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)
    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b2, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=l05, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)
    # 输入X: lv574_shared, shape[1,1,11008]
    b20 = sch.cache_read(block=b2, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=l04, preserve_unit_loops=True, index=-1)
    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=3
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b2, loop=l04)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    # print(sch.get(b20))
    l29, l30, l31, l32, l33, l331 = sch.get_loops(block=b20)
    l34, l35, l36, l37 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=l331, factors=[None, 128, 8, 4], preserve_unit_iters=True
    )
    sch.vectorize(loop=l37)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    sch.bind(loop=l36, thread_axis="threadIdx.y")
    
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    sch.bind(loop=l41, thread_axis="blockIdx.x")
    sch.bind(loop=l42, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)


sch_manual = tvm.tir.Schedule(ModuleToManual)
sch_manual.mod['main'] = sch_fused_decode6_fused_matmul7_add1(sch_manual.mod[func_name])
print(sch_manual.mod.script())
print("================================================")
rt_mod = tvm.build(sch_manual.mod, target="opencl")
print(rt_mod.imported_modules[0].get_source())

In [ ]:
# 耗时: 716400ms
# 代码结构调整， 更方便阅读
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    v7, v8, v9 = sch.sample_perfect_tile(
        # loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 128, 4]
    )
    # 8  128   4
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    v161, v162 = sch.sample_perfect_tile(loop=l16, n=2, max_innermost_factor=8, decision=[8, 43])
    l161, _ = sch.split(l16, [v161, v162], preserve_unit_iters=True)
    # 将reduce操作按照l161做拆分，以便并行化. 其过程是: 
    #   1. 申请 [l161, 1, 1, 4096]个xxx_rf内存
    #   2. 将计算拆为l161个chunks分别计算
    #   3. 按照l161维度reduce, 得到最终的输出
    sch.rfactor(l161, 0)
    ## 此时b1还是block('matmul'), 但是计算内容变为rf的合并操作(即: reduce = sum(sub_reduces))
    ## 此时拆分的reduce计算变为 block('matmul_rf')
    b2 = sch.get_block(name="matmul_rf", func_name = "main")
    # 调用rfactor后, 计算部分转移到了 block('matmul_rf'), 所以得重新获取loops
    # s = spatial,  r = reduce
    lb2s1, lb2s2, lb2s3, lb2r1, lb2r2, lb2r3, lb2r4 = sch.get_loops(block=b2)
    # 8, 128, 8, 43, 4, 8, 4
    sch.reorder(lb2s1, lb2s2, lb2r1, lb2r2, lb2r3, lb2r4, lb2s3)
    sch.bind(loop=lb2s1, thread_axis="blockIdx.x")
    sch.bind(loop=lb2s2, thread_axis="threadIdx.x")
    sch.bind(loop=lb2r1, thread_axis="threadIdx.y")

    sch.compute_inline(block=b0)
    b21 = sch.cache_write(block = b2, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block = b21, loop=lb2r1, preserve_unit_loops=True, index=-1)
    lb21, lb22, lb23, lb24, lb25, lb26, lb27 = sch.get_loops(block=b2)
    sch.vectorize(lb27)
    lb211, lb212, lb213, lb214, lb215,lb216, lb217 = sch.get_loops(block=b21)
    sch.vectorize(lb217)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b2, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=lb2r3, preserve_unit_loops=True, index=-1)
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)

    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b2, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=lb2r2, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)

    # 输入X: lv574_shared, shape[1,1,11008] -> [8, 128, 8, 1, 1, 11008/(8*128*8)] (无法整除)-> [8, 128, 8, 1, 1, 11008]
    b20 = sch.cache_read(block=b2, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=lb2r1, preserve_unit_loops=True, index=-1)

    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=2
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b2, loop=lb2r1)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    lb20l1, lb20l2, lb20l3, lb20l4, lb20l5, lb20l6 = sch.get_loops(block=b20)
    l34, l35, l36, l37 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=lb20l6, factors=[None, 128, 8, 4], preserve_unit_iters=True
    )
    # 这里 l37 = 4, 11008 / (threadIdx.x * threadIdx.y * 4) 无法整除, 所以只有一部分线程可以执行到shared_memory构建
    # 实际观测 sch.vectorize(loop=l37) 并没有生效，不知道是否因为不是所有线程都能执行, 所以取消了vload4
    sch.vectorize(loop=l37)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    sch.bind(loop=l36, thread_axis="threadIdx.y")
    
    #######################################################################################
    ###################以下为kenenl1: 执行 sub_reduce结果合并 & bias操作#######################
    #######################################################################################
    # 计算的中间结果: var_matmul_intermediate_rf, shape[8, 1,1,4096]
    #  8   128   4
    # l10, l11, l12
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    l411 = sch.fuse(l41,l42, l43)
    ## 调整记录:
    #       4, 256, 4 0.717400ms
    #       8, 128, 4 0.717368ms
    #       16, 64, 4 0.717368ms
    #       32, 32, 4 0.716808ms
    #       64, 16, 4 0.718056ms

    #       4, 128, 8 0.719936ms
    #       8, 64,  8 0.719024ms
    #       16, 32, 8 0.716400ms
    #       32, 16, 8 0.717544ms

    #       4, 64, 16 0.722680ms
    #       8, 32, 16 0.719608ms
    #       16, 16, 16 0.718744ms
    #       32, 8, 16 0.720376ms
    l411_1, l411_2, l411_3 = sch.split(l411, [16, 32, 8], preserve_unit_iters=True)
    sch.reverse_compute_at(block=b19, loop=l411_2, preserve_unit_loops=True, index=-1)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    # b19 对应block('matmul')的write部分，其实对应的是 block('T_add') write
    # 8 128 1 1 4
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    print(sch.get(l22))
    # 8 128 4 8
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    sch.vectorize(loop=l43)
    print("================================================================")
    print(sch.get(l41))
    sch.bind(loop=l41, thread_axis="blockIdx.x")
    sch.bind(loop=l42, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)

In [ ]:
# 耗时: 0.623664ms
# 操作: 调整工作组大小，每个线程计算4个输出 -> 2个输出
## ref to mlc-llm/dispatch/dispatch_tir_operator_adreno.py
def sch_fused_decode6_fused_matmul7_add1(func):
    sch = tvm.tir.Schedule(func)
    b0 = sch.get_block(name="decode", func_name="main")
    b1 = sch.get_block(name="matmul", func_name="main")
    # 1 1 4096 11008
    l2, l3, l4, l5 = sch.get_loops(block=b1)
    # l6 = 4096
    l6 = sch.fuse(l2, l3, l4, preserve_unit_iters=True)
    v7, v8, v9 = sch.sample_perfect_tile(
        loop=l6, n=3, max_innermost_factor=4, decision=[8, 256, 2]
    )
    # 8  128   4
    l10, l11, l12 = sch.split(loop=l6, factors=[v7, v8, v9], preserve_unit_iters=True)
    v13, v14, v15 = sch.sample_perfect_tile(
        loop=l5, n=3, max_innermost_factor=8, decision=[344, 4, 8]
    )
    l16, l17, l18 = sch.split(
        loop=l5, factors=[v13, v14, v15], preserve_unit_iters=True
    )
    ## 搜索记录:
    #       2, 172  0.939960ms
    #       4, 86   0.757256ms
    #       8, 43   716400ms
    v161, v162 = sch.sample_perfect_tile(loop=l16, n=2, max_innermost_factor=8, decision=[8, 43])
    l161, _ = sch.split(l16, [v161, v162], preserve_unit_iters=True)
    # 将reduce操作按照l161做拆分，以便并行化. 其过程是: 
    #   1. 申请 [l161, 1, 1, 4096]个xxx_rf内存
    #   2. 将计算拆为l161个chunks分别计算
    #   3. 按照l161维度reduce, 得到最终的输出
    sch.rfactor(l161, 0)
    ## 此时b1还是block('matmul'), 但是计算内容变为rf的合并操作(即: reduce = sum(sub_reduces))
    ## 此时拆分的reduce计算变为 block('matmul_rf')
    b2 = sch.get_block(name="matmul_rf", func_name = "main")
    # 调用rfactor后, 计算部分转移到了 block('matmul_rf'), 所以得重新获取loops
    # s = spatial,  r = reduce
    lb2s1, lb2s2, lb2s3, lb2r1, lb2r2, lb2r3, lb2r4 = sch.get_loops(block=b2)
    # 8, 128, 8, 43, 4, 8, 4
    sch.reorder(lb2s1, lb2s2, lb2r1, lb2r2, lb2r3, lb2r4, lb2s3)
    sch.bind(loop=lb2s1, thread_axis="blockIdx.x")
    sch.bind(loop=lb2s2, thread_axis="threadIdx.x")
    sch.bind(loop=lb2r1, thread_axis="threadIdx.y")

    sch.compute_inline(block=b0)
    b21 = sch.cache_write(block = b2, write_buffer_index=0, storage_scope="local")
    sch.reverse_compute_at(block = b21, loop=lb2r1, preserve_unit_loops=True, index=-1)
    lb21, lb22, lb23, lb24, lb25, lb26, lb27 = sch.get_loops(block=b2)
    sch.vectorize(lb27)
    lb211, lb212, lb213, lb214, lb215,lb216, lb217 = sch.get_loops(block=b21)
    sch.vectorize(lb217)

    # 权重W uint32: [1376, 4096]
    b80 = sch.cache_read(block=b2, read_buffer_index=1, storage_scope="local")
    sch.compute_at(block=b80, loop=lb2r3, preserve_unit_loops=True, index=-1)
    l801, l802, l803, l804, l805, l806, l807 = sch.get_loops(block=b80)
    sch.vectorize(l807)

    # 权重Scale fp16: [344, 4096]
    b81 = sch.cache_read(block=b2, read_buffer_index=2, storage_scope="local")
    sch.compute_at(block=b81, loop=lb2r2, preserve_unit_loops=True, index=-1)
    l811, l812, l813, l814, l815, l816 = sch.get_loops(block=b81)
    sch.vectorize(l816)

    # 输入X: lv574_shared, shape[1,1,11008] -> [8, 128, 8, 1, 1, 11008/(8*128*8)] (无法整除)-> [8, 128, 8, 1, 1, 11008]
    b20 = sch.cache_read(block=b2, read_buffer_index=0, storage_scope="shared")
    sch.compute_at(block=b20, loop=lb2r1, preserve_unit_loops=True, index=-1)

    v21 = sch.sample_categorical(
        candidates=[1, 2, 4, 8], probs=[0.25, 0.25, 0.25, 0.25], decision=2
    )
    sch.annotate(
        block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch", ann_val=v21
    )
    sch.vectorize(loop=l12)
    b27 = sch.decompose_reduction(block=b2, loop=lb2r1)
    sch.enter_postproc()
    sch.unannotate(block_or_loop=b20, ann_key="meta_schedule.cooperative_fetch")
    lb20l1, lb20l2, lb20l3, lb20l4, lb20l5, lb20l6 = sch.get_loops(block=b20)
    l34, l35, l36, l37 = sch.split(
        # loop=l33, factors=[None, 256, 8], preserve_unit_iters=True
        loop=lb20l6, factors=[None, 128, 8, 4], preserve_unit_iters=True
    )
    # 这里 l37 = 4, 11008 / (threadIdx.x * threadIdx.y * 4) 无法整除, 所以只有一部分线程可以执行到shared_memory构建
    # 实际观测 sch.vectorize(loop=l37) 并没有生效，不知道是否因为不是所有线程都能执行, 所以取消了vload4
    sch.vectorize(loop=l37)
    sch.bind(loop=l35, thread_axis="threadIdx.x")
    sch.bind(loop=l36, thread_axis="threadIdx.y")
    
    #######################################################################################
    ###################以下为kenenl1: 执行 sub_reduce结果合并 & bias操作#######################
    #######################################################################################
    # 计算的中间结果: var_matmul_intermediate_rf, shape[8, 1,1,4096]
    #  8   128   4
    # l10, l11, l12
    b19 = sch.cache_write(block=b1, write_buffer_index=0, storage_scope="local")
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    l411 = sch.fuse(l41,l42, l43)
    ## 调整记录:
    #       4, 256, 4 0.717400ms
    #       8, 128, 4 0.717368ms
    #       16, 64, 4 0.717368ms
    #       32, 32, 4 0.716808ms
    #       64, 16, 4 0.718056ms

    #       4, 128, 8 0.719936ms
    #       8, 64,  8 0.719024ms
    #       16, 32, 8 0.716400ms
    #       32, 16, 8 0.717544ms

    #       4, 64, 16 0.722680ms
    #       8, 32, 16 0.719608ms
    #       16, 16, 16 0.718744ms
    #       32, 8, 16 0.720376ms
    l411_1, l411_2, l411_3 = sch.split(l411, [16, 32, 8], preserve_unit_iters=True)
    sch.reverse_compute_at(block=b19, loop=l411_2, preserve_unit_loops=True, index=-1)
    b28 = sch.get_block(name="T_add", func_name="main")
    sch.reverse_compute_inline(block=b28)
    # b19 对应block('matmul')的write部分，其实对应的是 block('T_add') write
    # 8 128 1 1 4
    l22, l23, l24, l25, l26 = sch.get_loops(block=b19)
    sch.vectorize(loop=l26)
    print(sch.get(l22))
    # 8 128 4 8
    l41,l42,l43,l44 = sch.get_loops(block=b1)
    sch.vectorize(loop=l43)
    print("================================================================")
    print(sch.get(l41))
    sch.bind(loop=l41, thread_axis="blockIdx.x")
    sch.bind(loop=l42, thread_axis="threadIdx.x")
    return sch.mod["main"].with_attr("tir.is_scheduled", 1)